In [4]:
from utils import predict, data
import importlib
import nibabel as nib
from monai.transforms import Compose, LoadImaged, Orientationd, AsChannelLastd, ScaleIntensityd, Resized, AsChannelFirstd
from monai.transforms import Compose, LoadImage, Orientation, AsChannelLast, ScaleIntensity, Resize, AsChannelFirst, SplitDimd, ToNumpyd
import yaml
from yaml import Loader
import pandas as pd
import csv

In [ ]:
# importlib.reload(utils)
# utils.discover_files('/home/raffi/Projects/DeepDicomSort_MONAI/data/Brain-Tumor-Progression')

In [ ]:
i = 5
def gen(i):
    for k in range(i):
        yield k

In [ ]:
generator = gen(i)

In [ ]:
generator.__next__()

In [ ]:
path = "data/Brain-Tumor-Progression_nifti/B_Ax_T1_+C/PGBM-005.nii"
image = nib.load(path)
image.shape

In [ ]:
transforms = Compose([
        LoadImaged(keys=["image"], reader="NibabelReader", ensure_channel_first=True),
        Orientationd(keys=["image"], axcodes="RAS"),
        Resized(keys=["image"], spatial_size=[256, 256, 25], mode="trilinear"),
        SplitDimd(keys=["image"], dim=3, keepdim=False),
        ScaleIntensityd(keys=[f"image_{i}" for i in range(25)], minv=0, maxv=1, channel_wise=True),
        ToNumpyd(keys=[f"image_{i}" for i in range(25)])
    ])
output = transforms({"image": path})
display(output.keys())
output["image_0"].shape

In [56]:
importlib.reload(predict)
importlib.reload(data)
output = predict.run("data/Brain-Tumor-Progression", "SeriesDescription")
output["prediction"] = output["prediction"]
output["label"] = output["label"]
pd.set_option('display.max_rows', 500)
#display(pd.DataFrame(output))

output_df = predict.get_majority_vote(output)
display(output_df.head(20))
display(output_df.groupby("prediction").agg({"label": lambda x: list(x)}).reset_index())

regex = predict.construct_regex(output_df)
display(regex)

Found DICOM images
Data already converted.
Create dataset
Loading cached output
Constructing bidsmap


,label,prediction,count
5,ADC_reg,6,582
7,FLAIR_reg,0,395
13,Mask_Tumor,1,769
20,T1post,1,601
24,T1pre_reg,0,778
32,T2_reg,2,722
36,dT1,1,597
44,nCBF_reg,5,398
48,nRCBV_reg,1,392
56,sRCBV_reg,5,388


,prediction,label
0,0,"[FLAIR_reg, T1pre_reg]"
1,1,"[Mask_Tumor, T1post, dT1, nRCBV_reg]"
2,2,[T2_reg]
3,5,"[nCBF_reg, sRCBV_reg]"
4,6,[ADC_reg]


{0: '(FLAIR_reg)|(T1pre_reg)',
 1: '(Mask_Tumor)|(T1post)|(dT1)|(nRCBV_reg)',
 2: '(T2_reg)',
 5: '(nCBF_reg)|(sRCBV_reg)',
 6: '(ADC_reg)'}

In [55]:
importlib.reload(predict)
importlib.reload(data)
output = predict.run("data/Brain-Tumor-Progression", "SeriesDescription")


Found DICOM images
Data already converted.
Create dataset
Loading cached output
Constructing bidsmap


In [ ]:
importlib.reload(predict)
filter = predict.construct_regex(output_df)
display(filter)

In [ ]:
with open('config.yaml', 'r') as ymlfile:
    config = yaml.load(ymlfile, Loader=Loader)
config

In [ ]:
with open("/home/raffi/Projects/DeepDicomSortOwn/data/output/Predictions_model_all_brain_tumor_data.csv", newline='') as f:
    reader = csv.reader(f, delimiter="\t")
    output = list(reader)
display(output[1])

with open("/home/raffi/Projects/DeepDicomSortOwn/data/output/Predictions_model_all_brain_tumor_data.csv", newline='') as f:
    df = pd.read_csv(f, delimiter="\t")
display(df.head(20))